# AlphaJet - MVP3 
## Part #1 - Grab latest BTC price and create MAs, Bollinger bands and RSI features
#### Sources : 

##### ML for Algo Trading - Stefan Jansen book
##### Medium article "Retrieving Full Historical Data for Every Cryptocurrency on Binance & BitMex Using the Python API" from Peter Nistrup
https://betterprogramming.pub/easiest-way-to-use-the-bitmex-api-with-python-fbf66dc38633

In [1]:
import pandas_datareader.data as web
import pandas as pd
import numpy as np
import talib
import quandl
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import yfinance as yf
from datetime import date,timedelta, datetime
import math
import os.path
import time
from bitmex import bitmex
from binance.client import Client
from dateutil import parser
from tqdm import tqdm_notebook #(Optional, used for progress-bars)
#import pandas_profiling #Optional - for fancy data analysis reports!

#### Defining API keys for Binance and Bitmex

In [2]:
### API
bitmex_api_key = 'REDACTED'    #Enter your own API-key here
bitmex_api_secret = '[REDACTED]' #Enter your own API-secret here
binance_api_key = 'VaBzOEyR5SpRLCkWg2CBuBnlsAP6I0nEuVOFuO4AHO1IrDg7H3WNo2IizWBfCr03'    #Enter your own API-key here
binance_api_secret = 'lgDl4hN89eJ8P7BhhHKQqX9DBTrA3peK4fDgJj6L8wsdyonCYI1XJTOoEs60IyRO' #Enter your own API-secret here

#### Constants

In [3]:
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 750
#bitmex_client = bitmex(test=False, api_key=bitmex_api_key, api_secret=bitmex_api_secret)
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

### FUNCTIONS

In [4]:
def minutes_of_new_data(symbol, kline_size, data, source):
    if len(data) > 0:  old = parser.parse(data["timestamp"].iloc[-1])
    elif source == "binance": old = datetime.strptime('1 Jan 2017', '%d %b %Y')
    elif source == "bitmex": old = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=False).result()[0][0]['timestamp']
    if source == "binance": new = pd.to_datetime(binance_client.get_klines(symbol=symbol, interval=kline_size)[-1][0], unit='ms')
    if source == "bitmex": new = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=1, reverse=True).result()[0][0]['timestamp']
    return old, new

def get_all_binance(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "binance")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    if oldest_point == datetime.strptime('1 Jan 2017', '%d %b %Y'): print('Downloading all available %s data for %s. Be patient..!' % (kline_size, symbol))
    else: print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data.' % (delta_min, symbol, available_data, kline_size))
    klines = binance_client.get_historical_klines(symbol, kline_size, oldest_point.strftime("%d %b %Y %H:%M:%S"), newest_point.strftime("%d %b %Y %H:%M:%S"))
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
    if len(data_df) > 0:
        temp_df = pd.DataFrame(data)
        data_df = data_df.append(temp_df)
    else: data_df = data
    data_df.set_index('timestamp', inplace=True)
    if save: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

def get_all_bitmex(symbol, kline_size, save = False):
    filename = '%s-%s-data.csv' % (symbol, kline_size)
    if os.path.isfile(filename): data_df = pd.read_csv(filename)
    else: data_df = pd.DataFrame()
    oldest_point, newest_point = minutes_of_new_data(symbol, kline_size, data_df, source = "bitmex")
    delta_min = (newest_point - oldest_point).total_seconds()/60
    available_data = math.ceil(delta_min/binsizes[kline_size])
    rounds = math.ceil(available_data / batch_size)
    if rounds > 0:
        print('Downloading %d minutes of new data available for %s, i.e. %d instances of %s data in %d rounds.' % (delta_min, symbol, available_data, kline_size, rounds))
        for round_num in tqdm_notebook(range(rounds)):
            time.sleep(1)
            new_time = (oldest_point + timedelta(minutes = round_num * batch_size * binsizes[kline_size]))
            data = bitmex_client.Trade.Trade_getBucketed(symbol=symbol, binSize=kline_size, count=batch_size, startTime = new_time).result()[0]
            temp_df = pd.DataFrame(data)
            data_df = data_df.append(temp_df)
    data_df.set_index('timestamp', inplace=True)
    if save and rounds > 0: data_df.to_csv(filename)
    print('All caught up..!')
    return data_df

# 1) Features creation and Enriching

##### Getting the Data from Binance

In [5]:
data = get_all_binance('BTCUSDT','1h',save=True)

All caught up..!


In [6]:
data.head(3)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36525 entries, 2017-08-17 04:00:00.000 to 2021-10-14 14:00:00
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   open         36525 non-null  object
 1   high         36525 non-null  object
 2   low          36525 non-null  object
 3   close        36525 non-null  object
 4   volume       36525 non-null  object
 5   close_time   36525 non-null  int64 
 6   quote_av     36525 non-null  object
 7   trades       36525 non-null  int64 
 8   tb_base_av   36525 non-null  object
 9   tb_quote_av  36525 non-null  object
 10  ignore       36525 non-null  object
dtypes: int64(2), object(9)
memory usage: 3.3+ MB


In [7]:
data.tail(3)

,open,high,low,close,volume,close_time,quote_av,trades,tb_base_av,tb_quote_av,ignore
timestamp,,,,,,,,,,,
2021-10-14 12:00:00,57622.42000000,57986.20000000,57397.25000000,57563.02000000,2327.94242000,1634216399999,134204411.86624170,64422,1121.91173000,64682946.54358390,0
2021-10-14 13:00:00,57563.02000000,57760.00000000,57300.00000000,57399.71000000,1580.52184000,1634219999999,90896038.00221830,47955,763.35449000,43901859.92409220,0
2021-10-14 14:00:00,57399.72000000,57574.45000000,57391.24000000,57546.26000000,188.27408000,1634223599999,10828573.32135630,6897,112.47413000,6468865.93877150,0


##### using the last 30 days and cleaning up the original BTC file

In [8]:
#start_date = pd.to_datetime('5/1/2020 00:00', utc= True)
#end_date = pd.to_datetime('8/1/2021 00:00', utc= True)
#end_date = pd.to_datetime(datetime.now(), utc= True)
#start_date

In [9]:
# converting index to proper datetime object
data.index = pd.to_datetime(data.index)
data.index = data.index.tz_localize('utc')

In [10]:
#remove any duplicate
data = data.groupby(data.index).first()

In [28]:
#price = data[(data.index >= start_date) & (data.index <= end_date)]
price = data.tail(7600)

price=price[['open','high','low','close','volume']]


# Converts the header name to all uppercase
price.columns = [x.capitalize() for x in price.columns] 

# Add AdjClose column
price['AdjClose']=price['Close']

# Changing columns to float
price = price.astype(float)

# Renaming index
price.index.rename('Date', inplace=True)

price.head()

,Open,High,Low,Close,Volume,AdjClose
Date,,,,,,
2020-12-01 06:00:00+00:00,19483.73,19517.94,19309.87,19338.34,3129.776329,19338.34
2020-12-01 07:00:00+00:00,19338.33,19546.81,19300.00,19515.63,3009.225182,19515.63
2020-12-01 08:00:00+00:00,19515.62,19567.00,19441.19,19466.99,3143.172961,19466.99
2020-12-01 09:00:00+00:00,19467.00,19570.00,19426.96,19565.00,2824.268695,19565.00
2020-12-01 10:00:00+00:00,19564.99,19800.00,19558.77,19739.51,7640.260767,19739.51


In [29]:
price.dtypes

Open        float64
High        float64
Low         float64
Close       float64
Volume      float64
AdjClose    float64
dtype: object

#### Adding Moving Averages

In [30]:
feature_names = ['5d_close_pct']  # a list of the feature names for later

# Create moving averages and rsi for timeperiods of 5, 7, 14, 30, 50,100,200,350,700
for n in [5,7,14,20,30,50,100,200,350,600,700]:

    # Create the simple moving average indicator and divide by Adj_Close
    price['ma' + str(n)] = talib.SMA(price['AdjClose'].values, timeperiod=n) 
    price['tsma'+ str(n)] = talib.TSF(price['AdjClose'].values, timeperiod=n)
    price['tsma_norm'+ str(n)] = talib.TSF(price['AdjClose'].values, timeperiod=n)/price['AdjClose']
    
    # Create the exp moving average indicator and divide by Adj_Close
    price['ema' + str(n)] = talib.EMA(price['AdjClose'].values,timeperiod=n) 
    
    # Create the RSI indicator
    price['rsi' + str(n)] = talib.RSI(price['AdjClose'].values, timeperiod=n)
    
    # Add previous close price for the n previous days, and n-day percentage % price change
    price['previous'+ str(n)+'d_close'] = price['AdjClose'].shift(n) 
    price[str(n)+'d_close_pct'] = price['AdjClose'].pct_change(n)

    # Add rsi, moving average, tsma and previous days close prices and % change to the feature name list
    feature_names = feature_names + ['ma'+str(n),'tsma'+ str(n),'tsma_norm'+ str(n),'ema' + str(n), 'rsi' + str(n), 'previous'+str(n), str(n)+'d_close_pct']
    

print(feature_names)

price.tail()

['5d_close_pct', 'ma5', 'tsma5', 'tsma_norm5', 'ema5', 'rsi5', 'previous5', '5d_close_pct', 'ma7', 'tsma7', 'tsma_norm7', 'ema7', 'rsi7', 'previous7', '7d_close_pct', 'ma14', 'tsma14', 'tsma_norm14', 'ema14', 'rsi14', 'previous14', '14d_close_pct', 'ma20', 'tsma20', 'tsma_norm20', 'ema20', 'rsi20', 'previous20', '20d_close_pct', 'ma30', 'tsma30', 'tsma_norm30', 'ema30', 'rsi30', 'previous30', '30d_close_pct', 'ma50', 'tsma50', 'tsma_norm50', 'ema50', 'rsi50', 'previous50', '50d_close_pct', 'ma100', 'tsma100', 'tsma_norm100', 'ema100', 'rsi100', 'previous100', '100d_close_pct', 'ma200', 'tsma200', 'tsma_norm200', 'ema200', 'rsi200', 'previous200', '200d_close_pct', 'ma350', 'tsma350', 'tsma_norm350', 'ema350', 'rsi350', 'previous350', '350d_close_pct', 'ma600', 'tsma600', 'tsma_norm600', 'ema600', 'rsi600', 'previous600', '600d_close_pct', 'ma700', 'tsma700', 'tsma_norm700', 'ema700', 'rsi700', 'previous700', '700d_close_pct']


,Open,High,Low,Close,Volume,AdjClose,ma5,tsma5,tsma_norm5,ema5,...,rsi600,previous600d_close,600d_close_pct,ma700,tsma700,tsma_norm700,ema700,rsi700,previous700d_close,700d_close_pct
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-14 10:00:00+00:00,57414.60,57795.02,57240.85,57660.05,1614.86211,57660.05,57513.714,57519.900,0.997569,57585.859718,...,53.392451,48259.70,0.194787,48206.892143,54635.867662,0.947552,50066.394848,53.096473,46867.71,0.230272
2021-10-14 11:00:00+00:00,57660.04,58150.00,57552.49,57622.42,2290.50764,57622.42,57503.520,57718.638,1.001670,57598.046479,...,53.378026,48117.19,0.197543,48222.219357,54685.901651,0.949039,50087.952694,53.084106,46893.37,0.228797
2021-10-14 12:00:00+00:00,57622.42,57986.20,57397.25,57563.02,2327.94242,57563.02,57542.664,57670.884,1.001874,57586.370986,...,53.355234,48061.89,0.197685,48237.430829,54735.526684,0.950880,50109.279562,53.064568,46914.99,0.226964
2021-10-14 13:00:00+00:00,57563.02,57760.00,57300.00,57399.71,1580.52184,57399.71,57531.960,57493.917,1.001641,57524.150657,...,53.292565,47327.73,0.212814,48252.040643,54784.827822,0.954444,50130.079649,53.010848,47172.84,0.216796
2021-10-14 14:00:00+00:00,57399.72,57574.45,57391.24,57546.26,188.27408,57546.26,57558.292,57423.205,0.997862,57531.520438,...,53.341826,47274.52,0.217279,48267.026929,54834.505320,0.952877,50151.238509,53.053558,47055.86,0.222935


In [31]:
price.head()

,Open,High,Low,Close,Volume,AdjClose,ma5,tsma5,tsma_norm5,ema5,...,rsi600,previous600d_close,600d_close_pct,ma700,tsma700,tsma_norm700,ema700,rsi700,previous700d_close,700d_close_pct
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-01 06:00:00+00:00,19483.73,19517.94,19309.87,19338.34,3129.776329,19338.34,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-01 07:00:00+00:00,19338.33,19546.81,19300.00,19515.63,3009.225182,19515.63,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-01 08:00:00+00:00,19515.62,19567.00,19441.19,19466.99,3143.172961,19466.99,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-01 09:00:00+00:00,19467.00,19570.00,19426.96,19565.00,2824.268695,19565.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-01 10:00:00+00:00,19564.99,19800.00,19558.77,19739.51,7640.260767,19739.51,19525.094,19780.607,1.002082,19525.094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Adding PPO and Bollinger Bands

In [32]:
close = price['AdjClose'].values
up, mid, low = talib.BBANDS(close, timeperiod=14, nbdevup=2, nbdevdn=2, matype=0)

In [33]:
# calculate Bollinger bands for time period 14 days, using TAlib library 

up, mid, low = talib.BBANDS(close, timeperiod=14, nbdevup=2, nbdevdn=2, matype=0)
price['BB_up'] = up
price['BB_mid'] = mid
price['BB_low'] = low
price['BBP14'] = (price['AdjClose'] - low) / (up - low)
price['BB_up_norm'] = up/price['AdjClose']
price['BB_mid_norm'] = mid/price['AdjClose']
price['BB_low_norm'] = low/price['AdjClose']
price['BBP14_norm'] = price['BBP14']/price['AdjClose']


# Add ppo indicators

# create PPO Percentage Price Oscillator - normalized version of MACD for 20-day and 50-day exp moving average
price['ppo20-50'] = talib.PPO(price['AdjClose'].values, fastperiod=20, slowperiod=50, matype=0) 
price['ppo50-100'] = talib.PPO(price['AdjClose'].values, fastperiod=50, slowperiod=100, matype=0)
price['ppo7-20'] = talib.PPO(price['AdjClose'].values, fastperiod=7, slowperiod=20, matype=0) 
price['ppo100-200'] = talib.PPO(price['AdjClose'].values, fastperiod=100, slowperiod=200, matype=0) 
price['ppo200-350'] = talib.PPO(price['AdjClose'].values, fastperiod=200, slowperiod=350, matype=0) 
price['ppo100-700'] = talib.PPO(price['AdjClose'].values, fastperiod=100, slowperiod=700, matype=0) 
price['ppo20-200'] = talib.PPO(price['AdjClose'].values, fastperiod=20, slowperiod=200, matype=0) 
price['ppo50-350'] = talib.PPO(price['AdjClose'].values, fastperiod=50, slowperiod=350, matype=0) 

In [34]:
price.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'AdjClose', 'ma5', 'tsma5',
       'tsma_norm5', 'ema5', 'rsi5', 'previous5d_close', '5d_close_pct', 'ma7',
       'tsma7', 'tsma_norm7', 'ema7', 'rsi7', 'previous7d_close',
       '7d_close_pct', 'ma14', 'tsma14', 'tsma_norm14', 'ema14', 'rsi14',
       'previous14d_close', '14d_close_pct', 'ma20', 'tsma20', 'tsma_norm20',
       'ema20', 'rsi20', 'previous20d_close', '20d_close_pct', 'ma30',
       'tsma30', 'tsma_norm30', 'ema30', 'rsi30', 'previous30d_close',
       '30d_close_pct', 'ma50', 'tsma50', 'tsma_norm50', 'ema50', 'rsi50',
       'previous50d_close', '50d_close_pct', 'ma100', 'tsma100',
       'tsma_norm100', 'ema100', 'rsi100', 'previous100d_close',
       '100d_close_pct', 'ma200', 'tsma200', 'tsma_norm200', 'ema200',
       'rsi200', 'previous200d_close', '200d_close_pct', 'ma350', 'tsma350',
       'tsma_norm350', 'ema350', 'rsi350', 'previous350d_close',
       '350d_close_pct', 'ma600', 'tsma600', 'tsma_norm600', 'ema

In [35]:
price.tail(70)

,Open,High,Low,Close,Volume,AdjClose,ma5,tsma5,tsma_norm5,ema5,...,BB_low_norm,BBP14_norm,ppo20-50,ppo50-100,ppo7-20,ppo100-200,ppo200-350,ppo100-700,ppo20-200,ppo50-350
Date,,,,,,,,,,,,,,,,,,,,,
2021-10-11 17:00:00+00:00,57280.00,57562.73,57168.67,57418.99,1478.001130,57418.99,57371.988,57439.317,1.000354,57257.070136,...,0.974189,0.000015,1.370434,1.069460,1.268060,4.116643,7.412664,16.647671,6.672238,13.030484
2021-10-11 18:00:00+00:00,57418.99,57572.74,57201.00,57343.26,1439.182530,57343.26,57396.084,57311.889,0.999453,57285.800091,...,0.976303,0.000013,1.496071,1.096560,1.286901,4.090032,7.427363,16.682896,6.805779,13.047363
2021-10-11 19:00:00+00:00,57343.26,57839.04,57332.15,57488.33,1773.277637,57488.33,57404.152,57422.020,0.998847,57353.310060,...,0.974527,0.000014,1.656150,1.124157,1.292573,4.064295,7.440116,16.718069,6.976977,13.063684
2021-10-11 20:00:00+00:00,57488.33,57506.84,56800.00,57327.03,3094.673300,57327.03,57371.522,57420.542,1.001631,57344.550040,...,0.977962,0.000012,1.761945,1.154009,1.125445,4.032236,7.452163,16.744788,7.086922,13.074897
2021-10-11 21:00:00+00:00,57327.03,57400.00,56999.53,57156.54,1268.912270,57156.54,57346.830,57184.491,1.000489,57281.880027,...,0.981403,0.000010,1.834043,1.189787,0.890118,3.999352,7.463087,16.768313,7.166809,13.090631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-14 10:00:00+00:00,57414.60,57795.02,57240.85,57660.05,1614.862110,57660.05,57513.714,57519.900,0.997569,57585.859718,...,0.990008,0.000008,1.694439,0.193414,0.315441,1.910914,6.706768,16.991380,3.838183,8.956172
2021-10-14 11:00:00+00:00,57660.04,58150.00,57552.49,57622.42,2290.507640,57622.42,57503.520,57718.638,1.001670,57598.046479,...,0.990937,0.000007,1.798528,0.182854,0.098318,1.889982,6.669365,16.994708,3.912162,8.884132
2021-10-14 12:00:00+00:00,57622.42,57986.20,57397.25,57563.02,2327.942420,57563.02,57542.664,57670.884,1.001874,57586.370986,...,0.992611,0.000006,1.839651,0.160981,-0.055452,1.868706,6.634345,16.999251,3.909742,8.801897


## 2) Exporting to .csv

In [36]:
# Exporting last 3 weeks of data
Latest=price

# datetime object containing current date and time
now = datetime.now()

# YYYY-mm-dd_HM
date_string = now.strftime("%Y-%m-%d")
#Latest.to_csv("datasets_to_score/BTC_binance_"+str(date_string)+".csv", index=True, header=True)
Latest.to_csv("datasets_to_score/BTC_binance_"+str(date_string)+".csv", index=True, header=True)

print("'BTC_binance_"+str(date_string)+".csv' has been exported")



'BTC_binance_2021-10-14.csv' has been exported


In [24]:
Latest

,Open,High,Low,Close,Volume,AdjClose,ma5,tsma5,tsma_norm5,ema5,...,BB_low_norm,BBP14_norm,ppo20-50,ppo50-100,ppo7-20,ppo100-200,ppo200-350,ppo100-700,ppo20-200,ppo50-350
Date,,,,,,,,,,,,,,,,,,,,,
2020-12-30 14:00:00+00:00,27871.05,28184.44,27871.05,28182.98,2539.685081,28182.98,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 15:00:00+00:00,28182.98,28291.82,27984.19,28153.76,3130.428068,28153.76,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 16:00:00+00:00,28153.75,28240.37,27887.00,28153.78,3010.364220,28153.78,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 17:00:00+00:00,28153.77,28309.88,28050.00,28239.69,2913.702617,28239.69,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-12-30 18:00:00+00:00,28239.68,28530.00,28239.68,28510.86,3992.536754,28510.86,28248.214,28470.721,0.998592,28248.214000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-14 10:00:00+00:00,57414.60,57795.02,57240.85,57660.05,1614.862110,57660.05,57513.714,57519.900,0.997569,57585.859718,...,0.990008,0.000008,1.694439,0.193414,0.315441,1.910914,6.706768,16.991380,3.838183,8.956172
2021-10-14 11:00:00+00:00,57660.04,58150.00,57552.49,57622.42,2290.507640,57622.42,57503.520,57718.638,1.001670,57598.046479,...,0.990937,0.000007,1.798528,0.182854,0.098318,1.889982,6.669365,16.994708,3.912162,8.884132
2021-10-14 12:00:00+00:00,57622.42,57986.20,57397.25,57563.02,2327.942420,57563.02,57542.664,57670.884,1.001874,57586.370986,...,0.992611,0.000006,1.839651,0.160981,-0.055452,1.868706,6.634345,16.999251,3.909742,8.801897
